                                     Exploring Bitcoin Network

In [1]:
%matplotlib inline

In [2]:
import quandl as qd
qd.ApiConfig.api_key = "mYjt6YMKZ3yfFY5FT-KL"
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import praw
from urllib import request
from bs4 import BeautifulSoup
import datetime as dt
import dateutil.relativedelta as dr
import resource, sys
import pandas_datareader.data as web

In [3]:
# Retrieve S&P 500 Tickers
# https://stackoverflow.com/questions/13769278/are-there-any-free-apis-for-retrieving-the-sp-500s-component-symbols
def get_constituents():
        # URL request, URL opener, read content
        req = request.Request('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
        opener = request.urlopen(req)
        content = opener.read().decode() # Convert bytes to UTF-8

        soup = BeautifulSoup(content, 'lxml')
        tables = soup.find_all('table') # HTML table we actually need is tables[0] 

        external_class = tables[0].findAll('a', {'class':'external text'})

        tickers = []

        for ext in external_class:
            if not 'reports' in ext:
                tickers.append(ext.string)

        return tickers

In [4]:
tickrs = get_constituents()

In [5]:
def get_quandl_data(quandl_code, tickr):
    t_list = []
    for t in tickr:
        t_list.append(t)
        print(type(t_list), t_list)
        #API call to fetch data
        df = qd.get_table(quandl_code, qopts = {'columns': ['ticker', 'date', 'close']}, ticker=t_list, date = { 'gte': '2017-11-01', 'lte': '2017-11-02' }, paginate=True)
        
        # Remove element from the list
        #t_list.remove(t)
        
        #Save data as pickle
        #df.to_pickle('qd_data.pickle')
    return df

In [6]:
qd_data = get_quandl_data('WIKI/PRICES', tickrs)
qd_data

<class 'list'> ['MMM']


RecursionError: maximum recursion depth exceeded

In [7]:
d = {}
chunk_size = 100
for i in range(0,len(tickrs),chunk_size):
    chunk = tickrs[i:i+chunk_size]
    t_list = []
    for c in chunk:
        try:
            t_list.append(c)
            #API call to fetch data
            d[c] = qd.get_table('WIKI/PRICES', qopts = {'columns': ['ticker', 'date', 'close']}, ticker=t_list, date = { 'gte': '2016-11-01', 'lte': '2017-11-01' }, paginate=True)
        except Exception:
            pass
    

In [8]:
print(d)

{}


In [ ]:
# Using Pandas Data Reader 
start = dt.datetime(2017,1,1)
end = dt.datetime(2017,11,1)    
d = {}

for ticker in tickrs:
    try:
        d[ticker] = web.DataReader(ticker, 'yahoo', start, end)
    except Exception:
           pass


In [ ]:
pan = pd.Panel(d)
df1 = pan.minor_xs('Adj Close')
df1

In [15]:
df1.tail()

,MMM
Date,
2017-10-27,234.740005
2017-10-30,231.020004
2017-10-31,230.190002
2017-11-01,230.179993
2017-11-02,232.229996


In [ ]:
plt.subplots(figsize=(14,10))
plt.plot(qd_data.index, qd_data.Close)
plt.ylabel('Closing Price')
plt.title('Bitcoin Price (in USD)')

In [ ]:
# Reddit API Code
reddit = praw.Reddit(client_id='Xbd9h7VP6AU7cw',
                     client_secret='JuOtyE1MY2pdINsHL2terCm8C1U',
                     password='Cq130reddit',
                     user_agent='myRedditScriptApp0.1',
                     username='kevalshah90')
print(reddit.user.me())